In [6]:
ls ../data

001_A-B-chr1.EB         003_A-B-chr1.EB         test.cleanpileup
001_A-B-chr1.mutmatrix  003_A-B-chr1.mutmatrix  test.cutpileup
002.csv                 chr7.pilecount.gz       test.filterCount
002_A-B-chr1.EB         stop.file               test.pileup
002_A-B-chr1.mutmatrix  test.bed                test.pon


In [2]:
home = '/Users/mahtin'
testdata = f"{home}/Dropbox/Icke/Work/somVar/testdata"
pon_path = f"{testdata}/testpon"

### squeezing all data and shell paths into config

In [156]:
EBconfig = {
    "cleanpileup": "../shell/cleanpileup.mawk",
    "makeponlist": "../shell/makeponlist.sh",
    "csv2bed":"../shell/csv2bed.mawk",
    "pon2cols": "../shell/pon2cols.mawk",
    "pile2count": "../shell/pile2count2.mawk",
    "filterVar": "../shell/filterVar.mawk",
    "pon2tumor": "../shell/pon2tumor.mawk",
    "pon_path": pon_path,
    "genome_split": "/Users/mahtin/Dropbox/Icke/Work/static/genome/gatk/hg38/split",
    "MAPQ": 20,
    "Q": 25,
    "fit_pen": 0.5,
    "count_dict": {0:"alt+", 1:"alt-", 2:"depth+", 3:"depth-"}
}

In [89]:
from io import StringIO
import os
import sys
sys.path.append('../code')
from subprocess import PIPE, run

def bam2matrix(bam_file, mut_file, chrom, pon_list, EBconfig):
    '''
    converts the bam_file and the Pon-bams into a matrix for EB computations
    '''
    #unwrap the shell tools
    cleanpileup = EBconfig["cleanpileup"]
    makeponlist = EBconfig["makeponlist"]
    csv2bed = EBconfig["csv2bed"]
    pon2cols = EBconfig["pon2cols"]
    pile2count = EBconfig["pile2count"]
    filterVar = EBconfig["filterVar"]
    pon2tumor = EBconfig["pon2tumor"]
    gsplit = EBconfig["genome_split"]
    q = EBconfig["MAPQ"]
    Q = EBconfig["Q"]
    
    # create the bed file
    bed_file = f"{os.path.splitext(mut_file)[0]}_{chrom}.bed"
    run(f"{csv2bed} {mut_file} > {bed_file}", check=True, shell=True)
    
    # create the pon_list 
    pon_list = os.path.join(EBconfig['pon_path'], pon_list)
    pon_bam = f"{os.path.splitext(mut_file)[0]}_{chrom}.pon"
    run(f"{makeponlist} {bam_file} {pon_list} {pon_path} > {pon_bam}", check=True, shell=True)
    
    
    pileup_cmd = f"samtools mpileup -Q {Q} -q {q} -l {bed_file} -f {gsplit}/{chrom}.fa -b {pon_bam} -r {chrom}"
    process_cmd = f"cut -f $({pon2cols} {pon_list}) | {cleanpileup} | {pile2count} | {filterVar} {mut_file} {chrom} | {pon2tumor} 1"
    cmd = f"{pileup_cmd} | {process_cmd}"
    matrix_df = pd.read_csv(StringIO(
    run(cmd, stdout=PIPE, check=True, shell=True).stdout.decode('utf-8')), sep='\t')
    return matrix_df

In [90]:
bam_file = f"{testdata}/bam/002_A.bam"
mut_file = "../data/002.csv"
pon_list = "Pon_chr7short.txt"
chrom = "chr7"
df = bam2matrix(bam_file, mut_file, chrom, pon_list, EBconfig)
df[:3]

,Chr,Start,End,Ref,Alt,Tumor:Alt=Depth,PON:ALT=Depth
0,chr7,680651,680651,T,G,0-1=28-2,0|0|0|0-5|2|3|3=129|92|148|166-10|6|16|8
1,chr7,35670232,35670232,C,T,4-3=13-4,102|0|0|0-18|0|0|0=102|92|167|229-18|24|42|44
2,chr7,72826970,72826970,G,A,2-5=10-10,40|0|0|99-26|0|0|50=126|189|187|291-80|100|110...


In [91]:
row = df.iloc[3,:]
row

Chr                                                            chr7
Start                                                      75147946
End                                                        75147946
Ref                                                               C
Alt                                                               T
Tumor:Alt=Depth                                             1-2=9-9
PON:ALT=Depth      21|46|31|14-12|13|10|2=84|130|120|57-57|84|86|42
Name: 3, dtype: object

In [168]:
def get_count_df(pon_string, count_dict):
    '''
    converts the base-wise read coverage to a matrix
    '''
    
    data_split = [s for ad in pon_string.split("=") for s in ad.split("-")]
    count_df = pd.DataFrame.from_dict({dic[i]: v.split("|") for i,v in enumerate(data_split)})
    return count_df.astype(int)

In [169]:
cdf = get_count_df(row['PON:ALT=Depth'], EBconfig["count_dict"])
cdf

,alt+,alt-,depth+,depth-
0,21,12,84,57
1,46,13,130,84
2,31,10,120,86
3,14,2,57,42


# adapt the computation

## matrix to AB

In [134]:
from scipy.optimize import fmin_l_bfgs_b as minimize_func
from scipy.stats import chi2
from scipy.special import gammaln
import math

# the matrices for beta-binomial calculation
KS_matrix = np.array([[1, 0, 1, 1, 0, 1, 0, 0, 0], [
                     0, 1, -1, 0, 1, -1, 0, 0, 0]])
gamma_reduce = np.array([1, -1, -1, -1, 1, 1, 1, -1, -1])


def bb_loglikelihood(params, count_df):
    [a, b] = params
    ab_matrix = np.array([1, 1, 1, a + b, a, b, a + b, a, b])
    # convert df into matrix for np.array operations that change dims
    count_matrix = count_df.values
    # perform matrix multiplication to get inputs to log-gamma
    input_matrix = np.matmul(count_matrix, KS_matrix) + ab_matrix
    # get corresponding log-gamma values and reduce over pon-values
    # make sure, count_df is 2d to safe the check
    # if is_1d:  # check whether gammatrix is 2-dim - otherwise sum aggregation over axis 0 is faulty
    #     gamma_matrix = gammaln(input_matrix)
    # else:
    gamma_matrix = np.sum(gammaln(input_matrix), axis=0)
    # add or subtract using gamma_reduce matrix and sum to loglikelihood (scalar)
    log_likelihood = np.sum(gamma_matrix * gamma_reduce)
    return log_likelihood


def bb_loglikelihood_fitting(params, count_df, penalty):
    '''
    Fitting params [alpha, beta] to maximize loglikelihood
    '''

    # Here, we apply the penalty term of alpha and beta (default 0.5 is slightly arbitray...)
    result = penalty * \
        math.log(sum(params)) - bb_loglikelihood(params,
                                                count_df)  # matrix is dim2
    return result


def minimize(strand_count_df, pen):
    '''
    minimize the bb_loglikelihood to obtain AB params
    '''
    ab_params = minimize_func(
        bb_loglikelihood_fitting, [20, 20],
        args=(strand_count_df, pen), approx_grad=True,
        bounds=[(0.1, 10000000), (1, 10000000)]
    )[0]
    return [round(param, 5) for param in ab_params]
    
    
def fit_bb(count_df, pen):
    '''
    Obtaining maximum likelihood estimator of beta-binomial distribution
    count_df is the array of depth-mismatch (trials, success) pairs over the PoN list for either strand
    during minimization of fitting function (max for loglikelihood) penalty term is applied to constrain alpha and beta
        Ref for L-BFGS-B algorithm:
        A Limited Memory Algorithm for Bound Constrained Optimization
        R. H. Byrd, P. Lu and J. Nocedal. , (1995),
        SIAM Journal on Scientific and Statistical Computing, 16, 5, pp. 1190-1208.
    '''



    # get the respective control matrices (as dataframe) for positive and negative strands
    count_pos = count_df.loc[:, ['depth+', 'alt+']]
    count_neg = count_df.loc[:, ['depth-', 'alt-']]
    # minimize loglikelihood using L-BFGS-B algorithm
    ab_pos = minimize(count_pos, pen)
    ab_neg = minimize(count_neg, pen)
    return f"{ab_pos[0]}|{ab_pos[1]}-{ab_neg[0]}|{ab_neg[1]}"

In [135]:
bb_params = fit_bb(cdf, 0.5)
bb_params

'25.6636|65.04676-4.79774|29.78674'

In [136]:
def matrix2AB(row, config=None):
    count_df = get_count_df(row['PON:ALT=Depth'], config["count_dict"])
    return fit_bb(count_df, config['fit_pen'])

In [137]:
matrix2AB(row, config=EBconfig)

'25.6636|65.04676-4.79774|29.78674'

### expand matrix2AB to df

In [140]:
df['ABparams'] = df.apply(matrix2AB, config=EBconfig, axis=1)
df

,Chr,Start,End,Ref,Alt,Tumor:Alt=Depth,PON:ALT=Depth,AB,ABparams
0,chr7,680651,680651,T,G,0-1=28-2,0|0|0|0-5|2|3|3=129|92|148|166-10|6|16|8,0.1|2.30251-4.45091|8.45244,0.1|2.30251-4.45091|8.45244
1,chr7,35670232,35670232,C,T,4-3=13-4,102|0|0|0-18|0|0|0=102|92|167|229-18|24|42|44,0.1|1.0-0.1|1.0,0.1|1.0-0.1|1.0
2,chr7,72826970,72826970,G,A,2-5=10-10,40|0|0|99-26|0|0|50=126|189|187|291-80|100|110...,0.14402|1.0-0.16004|1.0,0.14402|1.0-0.16004|1.0
3,chr7,75147946,75147946,C,T,1-2=9-9,21|46|31|14-12|13|10|2=84|130|120|57-57|84|86|42,25.6636|65.04676-4.79774|29.78674,25.6636|65.04676-4.79774|29.78674
4,chr7,99722302,99722302,A,G,4-1=15-3,0|0|0|0-0|0|0|0=144|182|211|217-19|31|50|31,0.1|2.34352-0.1|1.97866,0.1|2.34352-0.1|1.97866
5,chr7,100958075,100958075,C,T,1-0=161-69,29|23|38|32-8|4|7|5=1096|1476|1547|1792-554|77...,10.72193|498.81024-4.58621|508.20524,10.72193|498.81024-4.58621|508.20524
6,chr7,100959332,100959332,C,G,4-0=43-25,23|32|43|44-2|2|1|0=281|354|342|486-91|149|161...,29.12842|268.39286-1.25987|104.27625,29.12842|268.39286-1.25987|104.27625
7,chr7,100959344,100959344,G,C,5-0=46-20,30|36|50|60-3|2|1|0=290|369|340|504-108|148|16...,45.30331|339.90614-1.04185|73.94303,45.30331|339.90614-1.04185|73.94303
8,chr7,101033998,101033998,G,C,16-0=70-32,139|0|188|0-22|0|15|0=536|588|796|690-301|395|...,0.11793|1.0-0.11982|1.52596,0.11793|1.0-0.11982|1.52596
9,chr7,101034004,101034004,G,A,17-0=70-33,142|0|187|0-24|0|16|0=553|579|803|691-299|414|...,0.11786|1.0-0.11808|1.43809,0.11786|1.0-0.11808|1.43809


## AB to EB

+ make a row with the same values as in EBold for comparison

In [309]:
row_dict = {
    "Chr": ["chr7"],
    "Start": 680651,
    "Tumor:Alt=Depth": "0-5=12-42",
    "ABparams": "0.20509|23.07746-0.22661|2.47235"
}

test_row = pd.DataFrame().from_dict(row_dict).iloc[0]
test_row

Chr                                            chr7
Start                                        680651
Tumor:Alt=Depth                           0-5=12-42
ABparams           0.20509|23.07746-0.22661|2.47235
Name: 0, dtype: object

In [314]:
def retrieveABdata(row, EBconfig):
    # retrieve the data from the row
    # target_s:
    # turn string "0-5=12-42" into target_s:
    # pd.Series
    # alt+       0
    # alt-       5
    # depth+    12
    # depth-    42
    target = row['Tumor:Alt=Depth']
    count_dict = EBconfig["count_dict"]
    target_split = [s for ad in target.split("=") for s in ad.split("-")]
    target_s = pd.Series({count_dict[i]:v for i,v in enumerate(target_split)}).astype(int)
    # params:
    # turn string A+|B+-A-|B- into flat AB list [A+, B+, A-, B-]
    params = row['ABparams']
    AB_list = [float(ab) for s in params.split("-") for ab in s.split("|")]
    return target_s, AB_list


def get_obs_df(target_s, cols):
    '''
    turn the target_s into obs_df
    '''
    
    # cols is either ['depth+', 'alt+'] or ['depth-', 'alt-']
    alt_type = cols[1]
    # creates an observation df for each observation from depth-alt to depth-depth
    n_minus_k = target_s[cols[0]] - target_s[alt_type]
    # obs_df is instantiated from target_s dict with n_minus_k + 1 rows
    obs_df = pd.DataFrame(target_s[cols].to_dict(), index=range(n_minus_k + 1))
    # alt column is incremented using index
    obs_df[alt_type] = obs_df[alt_type] + obs_df.index
    return obs_df


def bb_loglikelihood_1d(obs_row, params):
    '''
    specialized 1-d version of bb_loglikelihood for p_value of targets
    copy of code is justified by omitting one if clause in the heavily used 2-d version
    '''
    
    [a, b] = params
    ab_matrix = np.array([1, 1, 1, a + b, a, b, a + b, a, b])
    # convert df into matrix for np.array operations that change dims
    count_matrix = obs_row.values
    # perform matrix multiplication to get inputs to log-gamma
    input_matrix = np.matmul(count_matrix, KS_matrix) + ab_matrix
    # get corresponding log-gamma values and reduce over pon-values
    gamma_matrix = gammaln(input_matrix)
    # else:
    # gamma_matrix = np.sum(gammaln(input_matrix), axis=0)
    # add or subtract using gamma_reduce matrix and sum to loglikelihood (scalar)
    log_likelihood = np.sum(gamma_matrix * gamma_reduce)
    return log_likelihood


def bb_pvalue(obs_df, params):
    '''
    get the sum of exponentials of loglikelihoods (densities) per observation
    '''
    
    # get the loglikelihood per observation
    obs_df['p'] = obs_df.apply(bb_loglikelihood_1d, params=params_split[2:], axis=1)
    
    # sum up the exponentials
    p_value = np.exp(obs_df['p']).sum()

    return p_value

def fisher_combination(p_values):

    if 0 in p_values.values():
        return 0
    else:
        return 1 - chi2.cdf(sum([-2 * math.log(x) for x in p_values.values()]), 2 * len(p_values.values()))
    

def AB2EBscore(row, EBconfig):
    '''
    takes a df containing an AB column of shape "A+|A--B+|B-""
    '''
    # retrieve the data from the row
    target_s, AB_list = retrieveABdata(row, EBconfig)
    
    # get the p_values for each strand
    p_values = {}
    p_values['+'] = bb_pvalue(get_obs_df(target_s, ['depth+', 'alt+']), params_split[:2])
    p_values['-'] = bb_pvalue(get_obs_df(target_s, ['depth-', 'alt-']), params_split[2:])
    # combine p_values with fisher combination
    EB_p = fisher_combination(p_values)
    if EB_p < 1e-60:
        return 60
    if EB_p > 1.0 - 1e-10:
        return 0
    return -round(math.log10(EB_p), 3)

In [315]:
AB2EBscore(test_row, EBconfig)


0.234